In [ ]:
ВАШЕ_ИМЯ_И_ФАМИЛИЯ = ""

---

## Введение в обработку естественного языка. Наивная байесовская классификация

In [ ]:
import random
import os
from string import punctuation
from collections import defaultdict
from copy import deepcopy

Загружаем данные. У нас есть три выборки: обучающая (train), валидационная (dev) и тестовая (test). На валидационной выборке тестируем классификатор и подбираем гиперпараметры. Результаты работы классификатора на тестовой выборке проверяются на сервере СompAI, метки для нее не предоставляются.

In [ ]:
data_dir = '../FILIMDB'

In [ ]:
def load_data(file_name):
    """
    Reads specified file, returns list of strings
    :param file_name: file name in data_dir folder
    :returns list of strings
    """
    print('Loading %s' % file_name)
    data_path = os.path.join(data_dir, file_name)
    with open(data_path) as input_data:
        lines = input_data.readlines()
        lines = [l.strip() for l in lines]
    
    print('Loaded %d lines' % len(lines))
    return lines

In [ ]:
train_texts, train_labels = load_data('train.texts'), load_data('train.labels')
dev_texts, dev_labels = load_data('dev.texts'), load_data('dev.labels')
test_texts = load_data('test.texts')

Посмотрим на примеры негативных рецензий

In [ ]:
def sample_review(n, sentiment):
    for _ in range(n):
        idx = random.randrange(len(train_texts))
        while train_labels[idx] != sentiment:
            idx = random.randrange(len(train_texts))
        print(train_texts[idx])
        print("*" * 100)
    
sample_review(3, 'neg')

А также положительных

In [ ]:
sample_review(3, 'pos')

Предобработка данных -- дело творческое, и может довольно сильно повлиять на результат. 
Чаще всего она включает следующие этапы:
- Отделить пунктуацию от слов пробелами
- Вычистить все лишнее: специальные символы, остатки html-разметки и т.д. При этом обычно не надо вычищать буквы с акцентами (à, á...). 'Мусорные' символы лучше не удалять, а заменять на пробелы, иначе слова могут склеиться.
- Перевести буквы в нижний регистр

Затем текст разбивается на токены - отдельные слова, знаки пунктуации, числа и т.д.

Посмотрим, какие символы встречаются у нас в текстах, выберем из них те, которые мы хотим оставить

In [ ]:
char_set = set(ch for review in train_texts for ch in review)
sorted_chars = sorted(list(char_set))
print(''.join(sorted_chars))

Теперь вы можете написать функцию, которая предобрабатывает текст и разбивает каждый отзыв на токены.

Вам может пригодиться:

Методы str.strip() и str.split()

[Метод str.translate()](https://www.tutorialspoint.com/python/string_translate.htm)

Для более сложной обработки используйте [регулярные выражения](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F). В питоне для этого есть модуль [re](https://docs.python.org/3.1/library/re.html)

Подсказка. В python3 '\w' в регулярных выражениях обозначает строчную букву любого алфавита, цифру или подчеркивание. 
Например, re.findall('w+') найдет все токены, состоящие из этих символов.

In [ ]:
def tokenize(text):
    """
    Preprocesses text and split it into the list of words
    :param: text(str): movie review
    """
    # YOUR CODE HERE
    raise NotImplementedError()

tokenized_train_texts = [tokenize(r) for r in train_texts]
tokenized_dev_texts = [tokenize(r) for r in dev_texts]
tokenized_test_texts = [tokenize(r) for r in test_texts]

In [ ]:
# sanity check
assert all(len(x) > 0 for x in tokenized_train_texts)
unique_words = set(w for text in tokenized_train_texts for w in text)
for word in ["the", "and", "good"]:
    assert word in unique_words

Составим словарь из наиболее часто встречающихся в отзывах слов. Может быть полезно удалить из словаря стоп-слова - самые частые слова языка, не несущие важной информации для классификации (например "you", "the", "i" ...). Слова, не вошедшие в словарь, будем обозначать специальным символом "UNK"

Количество слов в словаре - важный гиперпараметр классификатора.

In [ ]:
VOCAB_SIZE = 8000

with open("stopwords.txt", "r") as f:
    STOPWORDS = map(lambda x: x.strip(), f.readlines())

STOPWORDS = set(STOPWORDS)

vocab = []

# YOUR CODE HERE
raise NotImplementedError()

vocab.append("UNK")
vocab = set(vocab) # for faster searching

In [ ]:
assert "good" in vocab
assert all(x not in vocab for x in STOPWORDS)
assert len(vocab) == VOCAB_SIZE + 1

Теперь векторизуем отзывы: создадим defaultdict для двух классов, содержащих число вхождений каждого слова. Для этого напишите функцию, которая принимает на вход обзор и текущий defaultdict некоторого класса, который она затем обновляет, исходя из слов, содержащихся в обзоре. Не забывайте о том, что все слова рассматриваемого документа, не вошедшие в словарь, должны считаться одним и тем же словом, обозначаемом как "UNK".

Выберите подходящий метод векторизации:

    bag-of-words - подсчитываем, сколько раз встретилось каждое слово
    binary bag-of-words - каждое слово в заданном документе надо учесть только один раз

In [ ]:
def update_class_word_counter(text, class_word_counter):
    # YOUR CODE HERE
    raise NotImplementedError()

positive_class_word_counter = defaultdict(int)
negative_class_word_counter = defaultdict(int)

for text, label in zip(tokenized_train_texts, train_labels):
    if label == 'neg':
        update_class_word_counter(text, negative_class_word_counter)
    else:
        update_class_word_counter(text, positive_class_word_counter)

In [ ]:
# sanity check
for counter in [positive_class_word_counter, negative_class_word_counter]:
    assert counter["think"] > 1000
    assert min(counter.values()) > 0
    assert all(isinstance(x, int) for x in counter.values())

Реализуйте наивный байесовский классификатор со сглаживанием (гиперпараметр ALPHA). Сохраните итоговые предсказания для обучающей, валидационной и тестовой выборок в переменные train_preds, dev_preds и test_preds соответственно

In [ ]:
ALPHA = 0.1

In [ ]:
def classify(texts, params=None):
    """
    Classify items
    :param texts: list of texts to classify
    :param params: any params you need.
    :return: list of predicted labels 
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
train_preds = classify(train_texts)
dev_preds = classify(dev_texts)
test_preds = classify(test_texts)

Проверим, что классификатор дает осмысленные предсказания на обучающей выборке - ошибка прогноза должна быть в районе 15%. На валидационной выборке ошибка может оказаться немного больше, чем на обучающей.

In [ ]:
train_score = sum([predict==label for predict, label in zip(train_preds, train_labels)]) / len(train_labels)
print("Train score: ", train_score)
dev_score = sum([predict==label for predict, label in zip(dev_preds, dev_labels)]) / len(dev_labels)
print("Development score: ", dev_score)

In [ ]:
assert(train_score > 0.82)

Создадим файл с прогнозами и отправим его на CompAI сервер. Для этого понадобится положить файл compai_ilimdb_sentiment.py, полученный при регистрации, в директорию с этим блокнотом. 

In [ ]:
res_file_name = "preds.tsv"
with open(res_file_name, 'w') as outp:
    for index, label in enumerate(train_preds):
        print('train/%d\t%s' % (index, label), file=outp)
    for index, label in enumerate(dev_preds):
        print('dev/%d\t%s' % (index, label), file=outp)
    for index, label in enumerate(test_preds):
        print('test/%d\t%s' % (index, label), file=outp)
print('Predictions saved to %s' % res_file_name)

In [ ]:
%run compai_ilimdb_sentiment.py submit $res_file_name

### Примеры способов улучшения модели классификации методом наивной байесовской классификации

* Изменить размер словаря

* Использовать биграммы помимо обычных слов

* Воспользоваться [стеммингом](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B5%D0%BC%D0%BC%D0%B8%D0%BD%D0%B3)

* Подобрать параметр, отвечающий за сглаживание

* Строить матрицу X на основе [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) признаков

* ???

* PROFIT!